In [ ]:
import sys
sys.path.append("../../")
from taxcalc import *
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt 
from matplotlib.transforms import BlendedGenericTransform
%matplotlib inline

In [8]:
CBO_growth = {2013: {'_factor_target': [0.022, 
                                        0.024,
                                        0.024,
                                        0.025,
                                        0.026,
                                        0.023,
                                        0.018,
                                        0.019,
                                        0.021,
                                        0.021,
                                        0.021,
                                        0.02,
                                        0.02,
                                        0.02]
                     }
              }

In [9]:
# Current law baseline
policy_cl = Policy()
behavior_cl = Behavior()
records_cl = Records("../../puf.csv")
calc_cl = Calculator(policy_cl, records_cl, behavior_cl)
calc_cl.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_cl.increment_year()
assert calc_cl.current_year == 2017
calc_cl.calc_all()
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [10]:
# CTC expansion
policy_t = Policy()
behavior_t = Behavior()
records_t = Records("../../puf.csv")
reform_t = {
    2017: {
        '_ACTC_Income_thd': [0],
        '_ACTC_rt': [0.45],
        '_II_credit': [[1000, 1000, 1000, 1000, 1000, 1000]],
        '_II_credit_ps': [[75000, 110000, 55000, 75000, 75000, 55000]],
        '_II_credit_prt': [0.05]
    }
}
policy_t.implement_reform(reform_t)
calc_t = Calculator(policy_t, records_t, behavior_t)
calc_t.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_t.increment_year()
assert calc_t.current_year == 2017
calc_t.calc_all()
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [ ]:
policy_repeal = Policy()
behavior_repeal = Behavior()
records_repeal = Records("../../puf.csv")
reform_repeal = {
    2017: {
        '_CTC_c': [0]
    }
}
policy_repeal.implement_reform(reform_repeal)
calc_repeal = Calculator(policy_repeal, records_repeal, behavior_repeal)
calc_repeal.growth.update_economic_growth(CBO_growth)
for i in range(4):
    calc_repeal.increment_year()
assert calc_repeal.current_year == 2017
calc_repeal.calc_all()
print "Done"

In [11]:
combined_cl = calc_cl.records._combined * calc_cl.records.s006
combined_t = calc_t.records._combined * calc_t.records.s006

diff = (combined_t - combined_cl).sum()
print diff
print diff / 10**9

10804531758.3
10.8045317583


In [12]:
def ten_years(calc_base, calc_ref):
    calc1 = copy.deepcopy(calc_base)
    calc2 = copy.deepcopy(calc_ref)
    diff = [0] * 10
    for i in range(10):
        comb1 = calc1.records._combined * calc1.records.s006
        comb2 = calc2.records._combined * calc2.records.s006
        diff[i] = (comb2 - comb1).sum() / 10**9
        if calc1.current_year < 2026:
            calc1.increment_year()
            calc2.increment_year()
            calc1.calc_all()
            calc2.calc_all()
    return diff

In [ ]:
path_repeal = ten_years(calc_cl, calc_repeal)
for x in path_repeal:
    print x
print '\n' + str(sum(path_repeal))

In [13]:
path_t = ten_years(calc_cl, calc_t)
for x in path_t:
    print x
print '\n' + str(sum(path_t))

10.8045317583
11.4799653631
11.7571512667
11.9056213031
12.3402499305
12.5738912182
12.8831154408
13.2659042151
13.2651190458
13.432599839

123.708149381


In [ ]:
use_ind = [i for i in range(219814) if calc_cl.records.n24[i] == 3 and calc_cl.records.c00100[i] < 250000]
totaltax = sum([calc_cl.records._iitax[i] * calc_cl.records.s006[i] for i in use_ind])
totalinc = sum([calc_cl.records.c00100[i] * calc_cl.records.s006[i] for i in use_ind])
avgRate = totaltax / totalinc
print avgRate
print totaltax / sum(calc_cl.records.s006)

In [ ]:
use_ind = [i for i in range(219814) if calc_t.records.n24[i] == 3 and calc_t.records.c00100[i] < 250000]
totaltax = sum([calc_t.records._iitax[i] * calc_t.records.s006[i] for i in use_ind])
totalinc = sum([calc_t.records.c00100[i] * calc_t.records.s006[i] for i in use_ind])
avgRate = totaltax / totalinc
print avgRate
print totaltax / sum(calc_t.records.s006)

In [ ]:
x1 = sum([calc_cl.records.s006[i] for i in range(219814) if (calc_cl.records._combined[i] < calc_repeal.records._combined[i])])
x2 = sum([calc_cl.records.s006[i] for i in range(219814) if (calc_cl.records._combined[i] > calc_t.records._combined[i]) and 
          (calc_cl.records.c00100[i] < 20000)])
print x1

